In [2]:
from openfermion.chem import MolecularData
from openfermionpyscf import run_pyscf
from mindquantum.core.gates import X
from mindquantum.core.circuit import Circuit
from mindquantum.core.operators import Hamiltonian
from mindquantum.simulator import Simulator
from mindquantum.algorithm.nisq import generate_uccsd
import mindspore as ms

ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")

In [3]:
dist = 1.545
geometry = [
    ["Li", [0.0, 0.0, 0.0 * dist]],
    ["H", [0.0, 0.0, 1.0 * dist]],
]
basis = "sto3g"
spin = 0
print("Geometry: \n", geometry)

Geometry: 
 [['Li', [0.0, 0.0, 0.0]], ['H', [0.0, 0.0, 1.545]]]


In [5]:
molecule_of = MolecularData(
    geometry,
    basis,
    multiplicity=2 * spin + 1
)
molecule_of = run_pyscf(
    molecule_of,
    run_scf=1,
    run_ccsd=1,
    run_fci=1
)

print("Hartree-Fock energy: %20.16f Ha" % (molecule_of.hf_energy))
print("CCSD energy: %20.16f Ha" % (molecule_of.ccsd_energy))
print("FCI energy: %20.16f Ha" % (molecule_of.fci_energy))

Hartree-Fock energy:  -7.8631473861153172 Ha
CCSD energy:  -7.8827512750262025 Ha
FCI energy:  -7.8827611661209840 Ha


In [6]:
molecule_of.save()
molecule_file = molecule_of.filename
print(molecule_file.split('/')[-1])

H1-Li1_sto3g_singlet


In [7]:
hartreefock_wfn_circuit = Circuit([X.on(i) for i in range(molecule_of.n_electrons)])
print(hartreefock_wfn_circuit)

      ┏━━━┓   
q0: ──┨╺╋╸┠───
      ┗━━━┛   
      ┏━━━┓   
q1: ──┨╺╋╸┠───
      ┗━━━┛   
      ┏━━━┓   
q2: ──┨╺╋╸┠───
      ┗━━━┛   
      ┏━━━┓   
q3: ──┨╺╋╸┠───
      ┗━━━┛   


In [34]:
ansatz_circuit, \
init_amplitudes, \
ansatz_parameter_names, \
hamiltonian_QubitOp, \
n_qubits, n_electrons = generate_uccsd(molecule_file, threshold=-1)

ccsd:-7.8827512750262025.
fci:-7.882761166120984.


In [10]:
total_circuit = hartreefock_wfn_circuit + ansatz_circuit
total_circuit.summary()
print("Number of parameters: %d" % (len(ansatz_parameter_names)))

                            Circuit Summary                            
╭──────────────────────┬──────────────────────────────────────────────╮
│ Info                 │ value                                        │
├──────────────────────┼──────────────────────────────────────────────┤
│ Number of qubit      │ 12                                           │
├──────────────────────┼──────────────────────────────────────────────┤
│ Total number of gate │ 12612                                        │
│ Barrier              │ 2560                                         │
│ Noise Channel        │ 0                                            │
│ Measurement          │ 0                                            │
├──────────────────────┼──────────────────────────────────────────────┤
│ Parameter gate       │ 640                                          │
│ 44 ansatz parameters │ p0, p8, p1, p9, p2, p10, p3, p11, p4, p12... │
╰──────────────────────┴──────────────────────────────────────────────╯

Number of parameters: 44


In [11]:
sim = Simulator('mqvector', total_circuit.n_qubits)
molecule_pqc = sim.get_expectation_with_grad(Hamiltonian(hamiltonian_QubitOp), total_circuit)

In [13]:
import numpy as np

n_params = len(total_circuit.params_name)
p0 = np.zeros(n_params)
f, g = molecule_pqc(p0)
print("Energy: ", f, "\nshape: ", f.shape, '\n')
print("Gradient: ", g, "\nshape: ", g.shape)

Energy:  [[-7.86314739+0.j]] 
shape:  (1, 1) 

Gradient:  [[[ 1.34471767e-10+0.j -8.63397351e-02+0.j  3.03317819e-08+0.j
   -5.01435225e-02+0.j -1.11986448e-14+0.j -3.92742288e-02+0.j
   -9.79661342e-15+0.j -9.49169980e-02+0.j  2.56777171e-15+0.j
   -3.92742288e-02+0.j  8.67624335e-15+0.j -9.49169980e-02+0.j
   -4.03653034e-11+0.j -3.14736683e-02+0.j  1.05200741e-09+0.j
   -4.93444721e-01+0.j -8.33135441e-04+0.j -1.67128982e-16+0.j
   -3.39336902e-17+0.j  1.64062196e-17+0.j  1.56089270e-16+0.j
    2.63247357e-17+0.j -2.29227406e-17+0.j  7.18235103e-03+0.j
    6.77141563e-03+0.j -1.21372104e-02+0.j -5.02920277e-17+0.j
    8.58644573e-18+0.j  1.35881050e-01+0.j -3.01404119e-02+0.j
   -3.06658683e-19+0.j  6.13317367e-19+0.j -1.01673171e-30+0.j
    6.40916648e-17+0.j -4.62884519e-17+0.j  3.63715716e-17+0.j
    1.43266077e-28+0.j  1.14782419e-16+0.j -3.01404119e-02+0.j
   -5.73451738e-17+0.j  5.60694299e-17+0.j -4.53927047e-17+0.j
   -2.38935339e-17+0.j -9.91196389e-04+0.j]]] 
shape:  (1, 1

In [15]:
def fun(p0, molecule_pqc, energy_list=None):
    f, g = molecule_pqc(p0)
    f = np.real(f)[0, 0]
    g = np.real(g)[0, 0]
    if energy_list is not None:
        energy_list.append(f)
        if len(energy_list) % 5 == 0:
            print(f"Step: {len(energy_list)},\tenergy: {f}")
    return f, g

fun(p0, molecule_pqc)

(-7.863147386117156,
 array([ 1.34471767e-10, -8.63397351e-02,  3.03317819e-08, -5.01435225e-02,
        -1.11986448e-14, -3.92742288e-02, -9.79661342e-15, -9.49169980e-02,
         2.56777171e-15, -3.92742288e-02,  8.67624335e-15, -9.49169980e-02,
        -4.03653034e-11, -3.14736683e-02,  1.05200741e-09, -4.93444721e-01,
        -8.33135441e-04, -1.67128982e-16, -3.39336902e-17,  1.64062196e-17,
         1.56089270e-16,  2.63247357e-17, -2.29227406e-17,  7.18235103e-03,
         6.77141563e-03, -1.21372104e-02, -5.02920277e-17,  8.58644573e-18,
         1.35881050e-01, -3.01404119e-02, -3.06658683e-19,  6.13317367e-19,
        -1.01673171e-30,  6.40916648e-17, -4.62884519e-17,  3.63715716e-17,
         1.43266077e-28,  1.14782419e-16, -3.01404119e-02, -5.73451738e-17,
         5.60694299e-17, -4.53927047e-17, -2.38935339e-17, -9.91196389e-04]))

In [16]:
from scipy.optimize import minimize

energy_list = []
res = minimize(fun, p0, args=(molecule_pqc, energy_list), method='bfgs', jac=True)

Step: 5,	energy: -7.880530580930981
Step: 10,	energy: -7.882174730572577
Step: 15,	energy: -7.882562727447891
Step: 20,	energy: -7.882708884709802
Step: 25,	energy: -7.882750900673263
Step: 30,	energy: -7.882751007241914
Step: 35,	energy: -7.882751007399819


In [17]:
print(f"Ground state: \n{res.fun}\n")
print(f"FCI: \n-7.882362286798721\n")
print(f"Optimized amplitudes: \n{res.x}")

Ground state: 
-7.882751007399819

FCI: 
-7.882362286798721

Optimized amplitudes: 
[ 2.34345856e-04  1.89400267e-03  3.63521253e-02  1.68776277e-02
  5.18249855e-10  9.10639091e-04  4.37222623e-10  1.38923003e-02
  4.11656882e-09  9.09882890e-04 -1.46346685e-09  1.38972754e-02
 -5.58182328e-04  4.76406239e-04  3.15054417e-03  5.52847047e-02
  2.19019780e-04 -7.44662476e-11  4.77309418e-08 -4.84776955e-08
 -1.25964213e-09 -2.33999442e-08  2.77919271e-08 -6.52959514e-05
  1.14983101e-04  9.11976219e-04 -2.38273173e-10  3.72805293e-10
 -5.71203426e-02  3.09812128e-03  1.61248235e-11 -7.33354169e-11
  7.22964899e-11 -4.01796399e-09  7.05605127e-08 -6.55434318e-08
  4.47025720e-14  2.07669387e-10  3.09813932e-03  3.97115612e-09
 -2.62600046e-07  2.48401094e-07 -8.03010633e-10  3.14520777e-04]
